In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install ultralytics
!pip install collections

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests
import os
def object_detection(img_folder):
  final_output = []
  for i in range(100):
    for j in range(4):
      img_path = img_folder + str(i) + '_' + str(j)  + '.png'
      image = Image.open(img_path)
      # you can specify the revision tag if you don't want the timm dependency
      processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
      model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

      inputs = processor(images=image, return_tensors="pt")
      outputs = model(**inputs)

      # convert outputs (bounding boxes and class logits) to COCO API
      # let's only keep detections with score > 0.9
      target_sizes = torch.tensor([image.size[::-1]])
      results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.7)[0]
      dict_ = {}
      detect_cls = []
      detect_box = []
      for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]

        detect_cls.append(model.config.id2label[label.item()])
        detect_box.append(box)
      dict_['detect cls'] = detect_cls
      dict_['detect box'] = detect_box
      final_output.append(dict_)
  return final_output

In [ ]:
path0 = '/content/drive/MyDrive/New_exp/Midjourney/quick_test/'
path1 = '/content/drive/MyDrive/New_exp/Midjourney/quick_test_ab1/'
path2 = '/content/drive/MyDrive/New_exp/Midjourney/quick_test_ab2/'

In [ ]:
import json
detect_result = object_detection(path0)
with open('/content/drive/MyDrive/Error_Detection/midj/quick_test.json', 'w') as f:
  json.dump(detect_result, f, indent=4)

In [ ]:
detect_result = object_detection(path1)
with open('/content/drive/MyDrive/Error_Detection/midj/quick_test_ab1.json', 'w') as f:
  json.dump(detect_result, f, indent=4)

In [ ]:
detect_result = object_detection(path2)
with open('/content/drive/MyDrive/Error_Detection/midj/quick_test_ab2.json', 'w') as f:
  json.dump(detect_result, f, indent=4)

KeyboardInterrupt: 